In [1]:
from clearml import Task
import numpy as np
import random
import os
import optuna

import joblib
import pandas as pd
import optuna.visualization as vis
import matplotlib.pyplot as plt

from pathlib import Path
from clearml import Logger
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

from src import data_preprocessing as dp
from src import pipeline as pl
from src import feature_engineering as fe
from src import feature_selection as fs
from src.model_evaluation import evaluate_multi_output
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold

SEED = 42 # vẫn phải chọn random của model đấy

# Python, NumPy
random.seed(SEED)
np.random.seed(SEED) 

# Hash seed cho Python interpreter (ảnh hưởng tới dict order)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Giới hạn luồng tính toán song song (để tránh floating-point nondeterminism)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# tạo sampler của Optuna có seed cố định
sampler = optuna.samplers.TPESampler(seed=SEED)


c:\Users\Admin\Documents\ML\group1\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Load raw Data
df = dp.load_data(r"data\raw data\Hanoi Daily 10 years.csv")


# 2. basic preprocessing for all data set
df = dp.basic_preprocessing(df=df)


 Loaded data with shape: (3660, 33)
Dropped column: 'description'
Dropped column: 'icon'
Dropped column: 'stations'
Dropped column: 'name'


In [ ]:
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

def step1_check_stationarity(df, target_column='temp'):
    """
    STEP 1: Kiểm tra stationarity của target variable và các features quan trọng
    """
    print("=" * 60)
    print("STEP 1: STATIONARITY CHECK")
    print("=" * 60)
    
    def adf_test(series, name):
        """Augmented Dickey-Fuller test"""
        result = adfuller(series.dropna())
        p_value = result[1]
        is_stationary = p_value < 0.05
        
        print(f"📊 {name:20} | p-value: {p_value:.6f} | {'✅ Stationary' if is_stationary else '❌ Non-stationary'}")
        return is_stationary, p_value
    
    # Kiểm tra target variable
    if target_column in df.columns:
        target_stationary, target_pvalue = adf_test(df[target_column], target_column)
    else:
        print(f"❌ Target column '{target_column}' not found")
        return
    
    # Kiểm tra các features quan trọng khác
    important_features = ['temp', 'tempmax', 'tempmin', 'dew', 
                          'humidity', 'sealevelpressure', 'feelslikemax', 
                          'feelslikemin', 'feelslike', 'precip',
                          'precipcover', 'windgust','windspeed','winddir'
                        'sealevelpressure','cloudcover','visibility','solarradiation',
                            'solarenergy','uvindex']
    
    print("\n🔍 CHECKING IMPORTANT FEATURES:")
    feature_results = {}
    
    for feature in important_features:
        if feature in df.columns and feature != target_column:
            is_stationary, p_value = adf_test(df[feature], feature)
            feature_results[feature] = {'stationary': is_stationary, 'p_value': p_value}
    
    # Summary
    print(f"\n📋 STATIONARITY SUMMARY:")
    non_stationary_features = [f for f, res in feature_results.items() if not res['stationary']]
    print(f"   Non-stationary features: {len(non_stationary_features)}/{len(feature_results)}")
    if non_stationary_features:
        print(f"   Features cần xử lý: {non_stationary_features}")
    
    return target_stationary, feature_results

# Chạy step 1
print("🔄 Running Step 1: Stationarity Check...")
target_stationary, feature_results = step1_check_stationarity(df, 'temp')

# đảm bảo data stationary

In [ ]:
def step2_analyze_trend_seasonality(df, target_column='temp'):
    """
    STEP 2: Phân tích trend và seasonality
    """
    print("\n" + "=" * 60)
    print("STEP 2: TREND & SEASONALITY ANALYSIS")
    print("=" * 60)
    
    if target_column not in df.columns:
        print(f"❌ Target column '{target_column}' not found")
        return
    
    series = df[target_column].dropna()
    
    # 1. Phân tích trend (rolling mean)
    rolling_30 = series.rolling(window=30, center=True).mean()
    rolling_365 = series.rolling(window=365, center=True).mean()
    
    # 2. Tính trend strength
    trend_strength_30 = rolling_30.std() / series.std() if series.std() > 0 else 0
    trend_strength_365 = rolling_365.std() / series.std() if series.std() > 0 else 0
    
    print(f"📈 TREND ANALYSIS:")
    print(f"   30-day trend strength:  {trend_strength_30:.4f}")
    print(f"   365-day trend strength: {trend_strength_365:.4f}")
    
    # 3. Phân tích seasonality (autocorrelation)
    print(f"\n🔄 SEASONALITY ANALYSIS:")
    
    # Monthly seasonality
    monthly_autocorr = series.autocorr(lag=30) if len(series) > 30 else series.autocorr(lag=7)
    yearly_autocorr = series.autocorr(lag=365) if len(series) > 365 else series.autocorr(lag=30)
    
    print(f"   Monthly autocorrelation (lag 30):  {monthly_autocorr:.4f}")
    print(f"   Yearly autocorrelation (lag 365): {yearly_autocorr:.4f}")
    
    # 4. Visualize (tuỳ chọn)
    try:
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 1, 1)
        plt.plot(series.index, series.values, alpha=0.7, label='Original')
        plt.plot(series.index, rolling_30.values, 'r-', linewidth=2, label='30-day Trend')
        plt.plot(series.index, rolling_365.values, 'g-', linewidth=2, label='365-day Trend')
        plt.title(f'Trend Analysis - {target_column}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 1, 2)
        lags = min(50, len(series)//4)
        pd.plotting.autocorrelation_plot(series.iloc[:1000] if len(series) > 1000 else series)
        plt.title('Autocorrelation Plot')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️  Visualization failed: {e}")
    
    return {
        'trend_strength_30': trend_strength_30,
        'trend_strength_365': trend_strength_365,
        'monthly_autocorr': monthly_autocorr,
        'yearly_autocorr': yearly_autocorr
    }

# Chạy step 2
print("🔄 Running Step 2: Trend & Seasonality Analysis...")
trend_seasonality_results = step2_analyze_trend_seasonality(df, 'temp')

🧩 1️⃣ Trend Analysis

30-day trend strength = 0.8851 → RẤT MẠNH
→ Có xu hướng dao động ngắn hạn rõ (theo tháng).

365-day trend strength = 0.0659 → YẾU
→ Không có xu hướng dài hạn (trong 10 năm không tăng/giảm nhiệt độ rõ rệt).

📊 => Không cần detrend theo năm.
Tín hiệu chính là seasonal (mùa vụ) chứ không phải trend dài hạn.

🧩 2️⃣ Seasonality Analysis

Monthly autocorr (lag 30) = 0.65

Yearly autocorr (lag 365) = 0.75

Cả hai đều cao và dương, nghĩa là dữ liệu rất có tính chu kỳ mạnh (lặp lại sau mỗi 12 tháng).
Biểu đồ autocorrelation của bạn cũng thể hiện rõ sóng lặp đều đặn theo năm.

📊 => Rất nên deseasonalize (loại bỏ yếu tố mùa vụ)
Trước khi huấn luyện model machine learning / regression / boosting, việc loại bỏ chu kỳ giúp:

Giảm autocorrelation trong residuals.

Làm model học tốt hơn các yếu tố phi chu kỳ (ví dụ hiệu ứng gió, mưa, độ ẩm…).

In [4]:

# 3. chia train, val, test (tỉ lệ 70/15/15)
train_size = 0.7
val_size = 0.15
n = len(df)

train_df = df.iloc[:int(train_size*n)]
val_df = df.iloc[int(train_size*n):int((train_size+val_size)*n)]
test_df = df.iloc[int((train_size+val_size)*n):]

# 4. Fit và transform trên preprocessing pipeline
pipeline1 = pl.build_preprocessing_pipeline()

# fit trên train
pipeline1.fit(train_df)

# transform 
train_processed = pipeline1.transform(train_df)
val_processed = pipeline1.transform(val_df)
test_processed = pipeline1.transform(test_df)


In [5]:
#Tạo feature engineering
train_feat, target_col = fe.feature_engineering(train_processed)
val_feat, _ = fe.feature_engineering(val_processed)
test_feat, _ = fe.feature_engineering(test_processed)
# print(train_feat.shape)

In [5]:
from src import feature_selection as fs
# for feat in train_feat.columns:
#    print(feat)
train_feat.head(1)

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,season_winter_roll_min_7,season_winter_roll_max_7,season_winter_roll_mean_10,season_winter_roll_std_10,season_winter_roll_min_10,season_winter_roll_max_10,season_winter_roll_mean_14,season_winter_roll_std_14,season_winter_roll_min_14,season_winter_roll_max_14
datetime,,,,,,,,,,,,,,,,,,,,,
2015-09-20,33.0,26.7,29.3,41.7,30.5,35.4,25.7,81.5,1.4,100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Chia X, y riêng biệt
X_train = train_feat.drop(columns= target_col)
y_train = train_feat[target_col]

X_val = val_feat.drop(columns= target_col)
y_val = val_feat[target_col]

X_test = test_feat.drop(columns= target_col)
y_test = test_feat[target_col]

print(f"Train: {X_train.shape, y_train.shape}, Val: {X_val.shape, y_val.shape}, Test: {X_test.shape, y_test.shape}")

Train: ((2562, 1430), (2562, 5)), Val: ((549, 1430), (549, 5)), Test: ((549, 1430), (549, 5))


In [ ]:
high = fs.DropHighlyCorrelated1(threshold= 0.8)
X_train = high.fit_transform(X_train)
X_val = high.transform(X_val)
X_test = high.transform(X_test)

# Chọn 10 features theo Lasso
selector = fs.FeatureSelector1(method='elasticnet', top_k=100, alpha=0.05)
X_train_sel = selector.fit_transform(X_train, y_train)
X_test_sel = selector.transform(X_test)
X_val_sel = selector.transform(X_val)


# Chọn 10 features theo Lasso
selector = fs.FeatureSelector1(method='lasso', top_k=60, alpha=0.05)
X_train_sel = selector.fit_transform(X_train, y_train)
X_test_sel = selector.transform(X_test)
X_val_sel = selector.transform(X_val)


In [ ]:
# 1. Tính phương sai từng feature
variances = X_train_sel.var().sort_values()
print("Top feature có phương sai thấp nhất:")
print(variances.head(10))

# 2. Tính tương quan giữa các feature
corr_matrix = X_train_sel.corr().abs()
high_corr = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
             .stack().sort_values(ascending=False))
print("Top cặp tương quan cao nhất:")
print(high_corr.head(10))

# 3. Kiểm tra mức độ đa dạng
unique_ratio = X_train_sel.nunique() / len(X_train_sel)
print("Tỷ lệ giá trị unique của các feature:")
print(unique_ratio.sort_values().head(10))


In [28]:
select2 = fs.FeatureSelectionGradientBoosting1(top_k= 60)
X_train_sel = select2.fit_transform(X_train, y_train)
X_val_sel = select2.transform(X_val)
X_test_sel = select2.transform(X_test)


In [7]:
select3 = pl.build_GB_featture_engineering_pipeline(top_k= 35)
X_train_sel = select3.fit_transform(X_train, y_train['temp_next_1'])
X_val_sel = select3.transform(X_val)
X_test_sel = select3.transform(X_test)

In [8]:
X_train_sel.to_csv(f"data/Old_X_train_sel.csv", index=False)
y_train.to_csv(f"data/Old_y_train.csv", index=False)
X_test_sel.to_csv(f"data/Old_X_test_sel.csv", index=False)
y_test.to_csv(f"data/Old_y_test.csv", index=False)
X_val_sel.to_csv(f"data/Old_X_val_sel.csv", index=False)
y_val.to_csv(f"data/Old_y_val.csv", index=False)

In [9]:
joblib.dump(pipeline1, "pipelines/preprocessing_pipeline.pkl")
joblib.dump(select3, "pipelines/featureSelection_pipeline.pkl")

['pipelines/featureSelection_pipeline.pkl']

In [9]:
select4 = fs.FeatureSelectionLGB(top_k=60)
X_train_sel = select4.fit_transform(X_train, y_train[['temp_next_1','temp_next_3']])
X_val_sel = select4.transform(X_val)
X_test_sel = select4.transform(X_test)

In [ ]:
select5 = fs.FeatureSelectionLGB(top_k=60)
X_train_sel = select5.fit_transform(X_train, y_train)
X_val_sel = select5.transform(X_val)
X_test_sel = select5.transform(X_test)

In [10]:
X_train_sel.columns

Index(['wind_chill', 'heat_index', 'temp_dew_interaction',
       'sunrise_roll_max_14', 'pressure_temp_index', 'sunrise_roll_max_7',
       'tempmin', 'winddir', 'windgust', 'solar_temp_index', 'windspeed',
       'feelslike', 'sunrise_roll_mean_14', 'day_cos_roll_mean_14',
       'temp_dew_interaction_roll_max_14', 'feelslikemax', 'sunrise', 'temp',
       'sunrise_roll_max_5', 'solarradiation', 'tempmin_roll_max_10',
       'tempmax', 'sealevelpressure', 'solar_temp_index_roll_max_14',
       'solar_temp_index_roll_std_14', 'sealevelpressure_roll_max_10',
       'temp_range', 'pressure_temp_index_roll_max_10', 'wind_temp_index',
       'heat_index_roll_max_10', 'wind_chill_roll_max_14', 'precipcover',
       'heat_index_roll_mean_10', 'winddir_roll_std_7',
       'windgust_roll_mean_14'],
      dtype='object')

In [ ]:
y_train.head(1)

,temp_next_1,temp_next_2,temp_next_3,temp_next_4,temp_next_5
datetime,,,,,
2015-09-20,26.8,25.9,28.0,29.9,30.5


In [ ]:
Task.current_task().close()

### Gradient Boosting 

In [ ]:
# mỗi lần chạy xong, thì tạo task mới
# Tạo task mới trong project Temperature Forecasting
# Task.current_task().close()
task_gb = Task.init(
    project_name="Temperature Forecasting",     # tên project (nếu chưa có sẽ tự tạo)
    task_name="lgb top 35, feature t", # tên task mới
    task_type=Task.TaskTypes.optimizer            # loại task (training / testing / optimizer ...)
)

# print("✅ Task created successfully!")
print("Task ID:", task_gb.id)

ClearML Task: created new task id=5124c5440edc449e894e8b41934fdffe
ClearML results page: https://app.clear.ml/projects/01d8ce929f0243de9a9d3adf48970730/experiments/5124c5440edc449e894e8b41934fdffe/output/log
Task ID: 5124c5440edc449e894e8b41934fdffe


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
2025-10-26 14:13:41,724 - clearml.storage - INFO - Uploading: 5.60MB from c:/Users/Admin/Documents/ML/group1/models/Tuning_lgb_model.pkl


██████████████████████████████████ 100% | 5.60/5.6 MB [00:22<00:00,  4.05s/MB]: 


In [27]:
task_grad = Task.get_task(task_id="5124c5440edc449e894e8b41934fdffe")
logger_grad = task_grad.get_logger()

In [20]:
select3 = pl.build_GB_featture_engineering_pipeline(top_k= 35)
X_train_sel = select3.fit_transform(X_train, y_train['temp_next_1'])
X_val_sel = select3.transform(X_val)
X_test_sel = select3.transform(X_test)


In [21]:

base_xgb = XGBRegressor(
    booster='gbtree',          # tương đương boosting_type='gbdt'
    learning_rate=0.0154,      # giống LGB
    max_depth=4,               # kiểm soát độ phức tạp cây
    n_estimators=417,          # số lượng cây
    subsample=0.91,            # tương đương subsample
    colsample_bytree=0.77,     # tương đương colsample_bytree
    reg_alpha=0.0071,          # L1 regularization
    reg_lambda=0.0143,         # L2 regularization
    min_child_weight=22,       # tương đương min_child_samples trong LGBM
    gamma=0.41,                # tương đương min_split_gain
    random_state=42,
    n_jobs=-1,
    tree_method='hist',        # nhanh, ổn với dataset cỡ nhỏ
    objective='reg:squarederror'
)

# Nếu bạn có nhiều output (5 ngày temp_next_1 → temp_next_5):

model_grad = MultiOutputRegressor(estimator=base_xgb, n_jobs=-1)

model_grad.fit(X_train_sel, y_train)

y_pred_train = model_grad.predict(X_train_sel)
y_pred_val = model_grad.predict(X_val_sel)
y_pred_test = model_grad.predict(X_test_sel)

In [11]:
tune_lgb = LGBMRegressor(
   boosting_type='gbdt',
    colsample_bytree=0.7688738230630878,
    learning_rate=0.015433744772417535,
    max_depth=4,
    min_child_samples=22,
    min_split_gain=0.4084759794499262,
    n_estimators=417,
    num_leaves=192,
    reg_alpha=0.00714911136031373,
    reg_lambda=0.014324694192079955,
    subsample=0.9098920662929666,
    random_state=42,
    n_jobs=-1
)

model_grad = MultiOutputRegressor(estimator=tune_lgb, n_jobs=-1)

model_grad.fit(X_train_sel, y_train)

y_pred_train = model_grad.predict(X_train_sel)
y_pred_val = model_grad.predict(X_val_sel)
y_pred_test = model_grad.predict(X_test_sel)   

# 3. Evaluate on validation
defGrad_val_metrics = evaluate_multi_output(y_val, y_pred_val)

print("Validation metrics (trung bình):", defGrad_val_metrics["average"])
print("Validation metrics (chi tiết từng ngày):", defGrad_val_metrics["per_day"])

# 4. Evaluate on test
defGrad_test_metrics = evaluate_multi_output(y_test, y_pred_test)
print("\nTest metrics (trung bình):", defGrad_test_metrics["average"])
print("Test metrics (chi tiết từng ngày):", defGrad_test_metrics["per_day"])


# 4. Evaluate on test
defGrad_train_metrics = evaluate_multi_output(y_train, y_pred_train)
print("\nTrain metrics (trung bình):", defGrad_train_metrics["average"])
print("Train metrics (chi tiết từng ngày):", defGrad_train_metrics["per_day"])


Validation metrics (trung bình): {'RMSE': 2.413410000484644, 'R2': 0.7706106187026828, 'MAPE': 8.892664484679358}
Validation metrics (chi tiết từng ngày): {'RMSE_day_1': 1.4873028184387793, 'R2_day_1': 0.9168395558615586, 'MAPE_day_1': 5.04995192833021, 'RMSE_day_2': 2.243727510443769, 'R2_day_2': 0.8106181970836153, 'MAPE_day_2': 8.11130681273522, 'RMSE_day_3': 2.608724514560317, 'R2_day_3': 0.7436475829081002, 'MAPE_day_3': 9.702965995386222, 'RMSE_day_4': 2.8095054528286774, 'R2_day_4': 0.7026341307525958, 'MAPE_day_4': 10.63039223639197, 'RMSE_day_5': 2.917789706151678, 'R2_day_5': 0.6793136269075444, 'MAPE_day_5': 10.96870545055317}

Test metrics (trung bình): {'RMSE': 1.878489595853664, 'R2': 0.8328467166011532, 'MAPE': 6.130599682403046}
Test metrics (chi tiết từng ngày): {'RMSE_day_1': 1.3298979174788912, 'R2_day_1': 0.9182826606318566, 'MAPE_day_1': 4.124902319336509, 'RMSE_day_2': 1.8386702907083567, 'R2_day_2': 0.8437629980828389, 'MAPE_day_2': 5.9745719530642685, 'RMSE_day_

In [22]:
# Đánh giá sơ bộ trước khi tune elasticnet, correl 1, 0.8, top30k

# 3. Evaluate on validation
defGrad_val_metrics = evaluate_multi_output(y_val, y_pred_val)

print("Validation metrics (trung bình):", defGrad_val_metrics["average"])
print("Validation metrics (chi tiết từng ngày):", defGrad_val_metrics["per_day"])

# 4. Evaluate on test
defGrad_test_metrics = evaluate_multi_output(y_test, y_pred_test)
print("\nTest metrics (trung bình):", defGrad_test_metrics["average"])
print("Test metrics (chi tiết từng ngày):", defGrad_test_metrics["per_day"])


# 4. Evaluate on test
defGrad_train_metrics = evaluate_multi_output(y_train, y_pred_train)
print("\nTrain metrics (trung bình):", defGrad_train_metrics["average"])
print("Train metrics (chi tiết từng ngày):", defGrad_train_metrics["per_day"])


Validation metrics (trung bình): {'RMSE': 2.399253411612038, 'R2': 0.7734292167364003, 'MAPE': 8.812086969793807}
Validation metrics (chi tiết từng ngày): {'RMSE_day_1': 1.4816105093889913, 'R2_day_1': 0.9174748926283436, 'MAPE_day_1': 5.0617301689946155, 'RMSE_day_2': 2.2298161767872564, 'R2_day_2': 0.8129592890298964, 'MAPE_day_2': 8.019040953218301, 'RMSE_day_3': 2.6065446246177344, 'R2_day_3': 0.7440758278786841, 'MAPE_day_3': 9.652890013710277, 'RMSE_day_4': 2.8113597541490867, 'R2_day_4': 0.7022414724004933, 'MAPE_day_4': 10.531836623561494, 'RMSE_day_5': 2.8669359931171203, 'R2_day_5': 0.6903946017445841, 'MAPE_day_5': 10.794937089484344}

Test metrics (trung bình): {'RMSE': 1.8567020200032274, 'R2': 0.8366284358568767, 'MAPE': 6.065753335306238}
Test metrics (chi tiết từng ngày): {'RMSE_day_1': 1.3064320973846781, 'R2_day_1': 0.9211409955646085, 'MAPE_day_1': 4.055765678252915, 'RMSE_day_2': 1.8118506903129292, 'R2_day_2': 0.8482876307895415, 'MAPE_day_2': 5.853063340891596, 'R

In [ ]:
# Validation metrics (trung bình): {'RMSE': 2.4817041064861445, 'R2': 0.7590955379469829, 'MAPE': 9.147749583976637}
# Validation metrics (chi tiết từng ngày): {'RMSE_day_1': 1.5812580327165688, 'R2_day_1': 0.9060009449623416, 'MAPE_day_1': 5.596094677035289, 'RMSE_day_2': 2.369307175034735, 'R2_day_2': 0.7888258473611602, 'MAPE_day_2': 8.520132535636671, 'RMSE_day_3': 2.6864010554618254, 'R2_day_3': 0.7281541691579952, 'MAPE_day_3': 9.949098491907808, 'RMSE_day_4': 2.8520942456203695, 'R2_day_4': 0.6935503665815363, 'MAPE_day_4': 10.679075692881625, 'RMSE_day_5': 2.9194600235972232, 'R2_day_5': 0.6789463616718809, 'MAPE_day_5': 10.994346522421793}

# Test metrics (trung bình): {'RMSE': 1.9783987429598198, 'R2': 0.8142301441998464, 'MAPE': 6.412545085748368}
# Test metrics (chi tiết từng ngày): {'RMSE_day_1': 1.3723285701062595, 'R2_day_1': 0.9129850618381672, 'MAPE_day_1': 4.296477004754045, 'RMSE_day_2': 1.9300379881007352, 'R2_day_2': 0.8278496566566587, 'MAPE_day_2': 6.202240635071937, 'RMSE_day_3': 2.161757510315994, 'R2_day_3': 0.7838824038741381, 'MAPE_day_3': 7.108353229679183, 'RMSE_day_4': 2.2571887287137495, 'R2_day_4': 0.7643736563627156, 'MAPE_day_4': 7.376897753353906, 'RMSE_day_5': 2.170680917562361, 'R2_day_5': 0.7820599422675529, 'MAPE_day_5': 7.0787568058827715}

In [ ]:
# dùng gb feature selection
# Validation metrics (trung bình): {'RMSE': 2.4418699884926363, 'R2': 0.765591019012981, 'MAPE': 8.88896073890469}
# Validation metrics (chi tiết từng ngày): {'RMSE_day_1': 1.4966404491973533, 'R2_day_1': 0.9157920770264553, 'MAPE_day_1': 5.136188928622812, 'RMSE_day_2': 2.3245514691120097, 'R2_day_2': 0.7967285645293968, 'MAPE_day_2': 8.23470815667883, 'RMSE_day_3': 2.6664461728907116, 'R2_day_3': 0.732177771044517, 'MAPE_day_3': 9.748483703459726, 'RMSE_day_4': 2.821701668774034, 'R2_day_4': 0.7000467641492176, 'MAPE_day_4': 10.433969898497432, 'RMSE_day_5': 2.9000101824890723, 'R2_day_5': 0.6832099183153186, 'MAPE_day_5': 10.89145300726464}

# Test metrics (trung bình): {'RMSE': 1.944220253739615, 'R2': 0.8207917793933074, 'MAPE': 6.322395592022064}
# Test metrics (chi tiết từng ngày): {'RMSE_day_1': 1.347442666845054, 'R2_day_1': 0.9161123175271811, 'MAPE_day_1': 4.178196794164813, 'RMSE_day_2': 1.937902886783428, 'R2_day_2': 0.8264437737903529, 'MAPE_day_2': 6.229555734228328, 'RMSE_day_3': 2.114146717995285, 'R2_day_3': 0.7932971703781171, 'MAPE_day_3': 6.952601054004568, 'RMSE_day_4': 2.161026970236575, 'R2_day_4': 0.7840225186659064, 'MAPE_day_4': 7.144809054211748, 'RMSE_day_5': 2.1605820268377327, 'R2_day_5': 0.7840831166049791, 'MAPE_day_5': 7.106815323500857}

In [28]:

# 5. Save model 
model_path = r'models/Top35_Not_tune_selecGB40_LGB_model.pkl' 
joblib.dump(model_grad, model_path)
print(f"no_feature_t_XGB_selec_gb1 model saved to {model_path}")

# 6. Log lên ClearML
val_avg_metrics = defGrad_val_metrics["average"]
test_avg_metrics = defGrad_test_metrics["average"]
train_avg_metrics = defGrad_train_metrics["average"]

logger_grad.report_scalar("Default Metrics", "RMSE_val_avg", val_avg_metrics["RMSE"], iteration=0)
logger_grad.report_scalar("Default Metrics", "R2_val_avg", val_avg_metrics["R2"], iteration=0)
logger_grad.report_scalar("Default Metrics", "MAPE_val_avg", val_avg_metrics["MAPE"], iteration=0)
logger_grad.report_scalar("Default Metrics", "RMSE_test_avg", test_avg_metrics["RMSE"], iteration=0)
logger_grad.report_scalar("Default Metrics", "R2_test_avg", test_avg_metrics["R2"], iteration=0)
logger_grad.report_scalar("Default Metrics", "MAPE_test_avg", test_avg_metrics["MAPE"], iteration=0)
logger_grad.report_scalar("Default Metrics", "RMSE_train_avg", train_avg_metrics["RMSE"], iteration=0)
logger_grad.report_scalar("Default Metrics", "R2_train_avg", train_avg_metrics["R2"], iteration=0)
logger_grad.report_scalar("Default Metrics", "MAPE_train_avg", train_avg_metrics["MAPE"], iteration=0)

task_grad.upload_artifact("Default Metrics Detailed", {
    "Validation": defGrad_val_metrics,
    "Test": defGrad_test_metrics,
    "Train": defGrad_train_metrics
})


no_feature_t_XGB_selec_gb1 model saved to models/Top35_Not_tune_selecGB40_LGB_model.pkl


True

In [34]:
## B. Hyperparameter tuning cho LGB
def objective(trial):
    # === 1. booster trước, để dùng điều kiện ===
    boosting_type = trial.suggest_categorical('boosting_type', ['gbdt', 'dart'])

    # === 2. Không gian hyperparameter cho LGBM ===
    params = {
        'boosting_type': boosting_type,
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.3),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 7),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_bin': trial.suggest_int('max_bin', 64, 512),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),

        # Fixed parameters cho LGBM
        'objective': 'regression',
        'metric': 'rmse',
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }

    # --- Thêm các tham số riêng cho DART ---
    if boosting_type == 'dart':
        params['drop_rate'] = trial.suggest_float('drop_rate', 0.05, 0.5)
        params['skip_drop'] = trial.suggest_float('skip_drop', 0.3, 0.7)

    # === 3. TimeSeriesSplit CV===
    cv = TimeSeriesSplit(n_splits=5)
    rmse_scores = []

    for train_idx, val_idx in cv.split(X_train_sel):
        X_tr, X_val = X_train_sel.iloc[train_idx], X_train_sel.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # === 4. Train model ===
        base_model = LGBMRegressor(**params)
        model = MultiOutputRegressor(estimator=base_model, n_jobs=-1)

        model.fit(X_tr, y_tr)
        y_pred_val = model.predict(X_val)

        # Dùng evaluate_multi_output để tính RMSE trung bình
        metrics = evaluate_multi_output(y_val, y_pred_val)
        rmse_scores.append(metrics["average"]["RMSE"])

    mean_rmse = np.mean(rmse_scores)

    # === 5. Log kết quả lên ClearML ===
    logger_grad.report_scalar(
        title='Optuna Tuning - LGB',
        series='Trial Avg RMSE',
        value=mean_rmse,
        iteration=trial.number
    )

    params_table = pd.DataFrame([params], index=[0])
    logger_grad.report_table(
        title=f"Trial {trial.number} Parameters",
        series="params",
        iteration=trial.number,
        table_plot=params_table
    )

    # === 6. Trả về RMSE trung bình để Optuna minimize ===
    return mean_rmse


In [ ]:
# # Try new objective with pruning
# def objective(trial):
#     # === 1. Định nghĩa không gian hyperparameter thông minh hơn ===
#     params = {

#         # Sử dụng log=True để tìm kiếm learning_rate hiệu quả hơn
#         'loss': trial.suggest_categorical('loss', ['squared_error', 'huber']),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1500), # Mở rộng khoảng một chút
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05), # Chia nhỏ bước nhảy
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
#         'max_depth': trial.suggest_int('max_depth', 3, 8),
#         'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.2),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']), # Bỏ None để giảm độ phức tạp
#         'tol': 1e-4,
#         # 'ccp_alpha': 0.0,
#         'random_state': 42
#     }

#     # Chỉ gợi ý 'alpha' khi 'loss' là 'huber'
#     if params['loss'] == 'huber':
#         params['alpha'] = trial.suggest_float('alpha', 0.8, 0.99)

#     # === 2. TimeSeriesSplit CV với PRUNING (CẮT TỈA) ===
#     cv = TimeSeriesSplit(n_splits=5)
#     rmse_scores = []

#     # Dùng enumerate để lấy step cho pruning
#     for step, (train_idx, val_idx) in enumerate(cv.split(X_train)):
#         X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#         y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#         # === 3. Train model ===
#         model = GradientBoostingRegressor(**params)
#         model.fit(X_tr, y_tr)
#         y_pred_val = model.predict(X_val)

#         metrics = evaluate(y_val, y_pred_val)
#         rmse_score = metrics['RMSE']
#         rmse_scores.append(rmse_score)
        
#         # === BƯỚC CẢI TIẾN QUAN TRỌNG NHẤT: PRUNING ===
#         # 1. Báo cáo kết quả trung gian của fold này cho Optuna
#         trial.report(rmse_score, step)

#         # 2. Kiểm tra xem trial này có nên bị dừng sớm không
#         if trial.should_prune():
#             # Ném ra ngoại lệ để dừng trial này và chuyển sang trial tiếp theo
#             raise optuna.exceptions.TrialPruned()

#     mean_rmse = np.mean(rmse_scores)

#     # === 4. Log kết quả lên ClearML (Giữ nguyên) ===
#     logger_grad.report_scalar(
#         title='Optuna Tuning - GradientBoosting', series='Trial RMSE',
#         value=mean_rmse, iteration=trial.number)
    
#     params_table = pd.DataFrame([{"trial": trial.number, "mean_rmse": mean_rmse, **trial.params}])
#     logger_grad.report_table(
#         title="All Trials Summary", series="params_summary",
#         iteration=trial.number, table_plot=params_table)

#     # === 5. Trả về RMSE trung bình để Optuna minimize ===
#     return mean_rmse

In [ ]:
# def objective_r2(trial):
#     # === 1. Định nghĩa không gian hyperparameter (Giữ nguyên hoặc tinh chỉnh nếu muốn) ===
#     # Sử dụng không gian tìm kiếm đã được tinh chỉnh ở lần trước
#     params = {
#         'objective': 'regression',
#         'metric': 'rmse',
#         'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 31, 256),
#         'max_depth': trial.suggest_int('max_depth', 3, 15),
#         'min_child_samples': trial.suggest_int('min_child_samples', 10, 80),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 1.0, log=True),
#         'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.5),
#         'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
#         'random_state': 42,
#         'n_jobs': -1
#     }


#     # === 2. TimeSeriesSplit CV ===
#     cv = TimeSeriesSplit(n_splits=5)
#     r2_scores = [] # THAY ĐỔI 2: Đổi tên list để lưu R-squared scores

#     for train_idx, val_idx in cv.split(X_train_sel):
#         X_tr, X_val = X_train_sel.iloc[train_idx], X_train_sel.iloc[val_idx]
#         y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#         # === 3. Train model ===
#         model = LGBMRegressor(**params)
#         model.fit(X_tr, y_tr)
#         y_pred_val = model.predict(X_val)

#         metrics = evaluate(y_val, y_pred_val)
#         r2_score = metrics['R2']  # THAY ĐỔI 3: Lấy R-squared thay vì RMSE
#         r2_scores.append(r2_score)

#     mean_r2 = np.mean(r2_scores) # THAY ĐỔI 4: Tính trung bình R-squared

#     # === 4. Log kết quả lên ClearML (Cập nhật để log R-squared) ===
#     logger_grad.report_scalar(
#         title='Optuna Tuning - R-squared', # Cập nhật title
#         series='Trial R2',               # Cập nhật series
#         value=mean_r2,                   # Log giá trị R-squared
#         iteration=trial.number
#     )

#     params_summary = {"trial": trial.number, "mean_r2": mean_r2, **trial.params}
#     logger_grad.report_table(
#         title="All Trials Summary (R2)", series="params_summary_r2",
#         iteration=trial.number, table_plot=pd.DataFrame([params_summary])
#     )

#     # === 5. Trả về R-squared trung bình để Optuna MAXIMIZE ===
#     return mean_r2

In [ ]:
# # === 3. Chạy Optuna study cho Gradient Boosting ===
# #maximize R-squared
# study_grad = optuna.create_study(
#     direction='maximize',  # THAY ĐỔI: Tối đa hóa R-squared
#     sampler=sampler
# )

# # Tiến hành tối ưu
# study_grad.optimize(objective_r2, n_trials=50, show_progress_bar=True)

# # === 4. In kết quả tốt nhất ===
# print("Best parameters:", study_grad.best_trial.params)
# print("Best R-squared:", study_grad.best_value)

In [35]:
# === 3. Chạy Optuna cho Gradient Boosting ===
study_grad = optuna.create_study(
    direction='minimize',
    sampler=sampler
)
study_grad.optimize(objective, n_trials=50, show_progress_bar=True)

# === 4. In kết quả tốt nhất (Không đổi) ===
print("Best parameters:", study_grad.best_trial.params)
print("Best R2 (from Optuna CV):", study_grad.best_value)
# task_grad.upload_artifact("Best Parameters", study_grad.best_trial.params)

[I 2025-10-26 13:52:20,943] A new study created in memory with name: no-name-dfd78571-36f8-4f02-a365-81020f830332
Best trial: 0. Best value: 2.42558:   2%|▏         | 1/50 [01:02<51:18, 62.82s/it]

[I 2025-10-26 13:53:23,754] Trial 0 finished with value: 2.42558174126523 and parameters: {'boosting_type': 'dart', 'max_depth': 10, 'min_child_weight': 5.727904470799623, 'min_split_gain': 0.26844820512829465, 'learning_rate': 0.04537761219144586, 'reg_alpha': 0.40679084943595434, 'reg_lambda': 2.769889922756281e-05, 'subsample': 0.6783931449676581, 'colsample_bytree': 0.6180909155642152, 'subsample_freq': 3, 'n_estimators': 644, 'num_leaves': 55, 'max_bin': 436, 'feature_fraction': 0.7427013306774357, 'drop_rate': 0.17642052935932134, 'skip_drop': 0.5170784332632994}. Best is trial 0 with value: 2.42558174126523.


Best trial: 0. Best value: 2.42558:   4%|▍         | 2/50 [01:09<23:57, 29.95s/it]

[I 2025-10-26 13:53:30,705] Trial 1 finished with value: 14.311230172634913 and parameters: {'boosting_type': 'dart', 'max_depth': 3, 'min_child_weight': 8.862326508576253, 'min_split_gain': 0.23167343078899721, 'learning_rate': 0.01040697346842839, 'reg_alpha': 1.0656401760606452e-05, 'reg_lambda': 0.11948328168545447, 'subsample': 0.8827429375390468, 'colsample_bytree': 0.8916028672163949, 'subsample_freq': 6, 'n_estimators': 203, 'num_leaves': 66, 'max_bin': 116, 'feature_fraction': 0.9452413703502374, 'drop_rate': 0.3304841570724011, 'skip_drop': 0.43235920994105964}. Best is trial 0 with value: 2.42558174126523.


Best trial: 0. Best value: 2.42558:   6%|▌         | 3/50 [02:49<48:27, 61.85s/it]

...

[I 2025-10-26 14:10:23,380] Trial 47 finished with value: 2.2706015366636993 and parameters: {'boosting_type': 'gbdt', 'max_depth': 3, 'min_child_weight': 0.07124338182311717, 'min_split_gain': 0.03138603342859314, 'learning_rate': 0.005005269780499635, 'reg_alpha': 0.0005112960078429123, 'reg_lambda': 0.00314450270072662, 'subsample': 0.7598721011918322, 'colsample_bytree': 0.6938005639735986, 'subsample_freq': 4, 'n_estimators': 830, 'num_leaves': 147, 'max_bin': 331, 'feature_fraction': 0.8551373600887916}. Best is trial 38 with value: 2.263787859316132.


Best trial: 38. Best value: 2.26379:  98%|█████████▊| 49/50 [18:37<00:18, 18.49s/it]

[I 2025-10-26 14:10:58,298] Trial 48 finished with value: 3.0027868783875338 and parameters: {'boosting_type': 'dart', 'max_depth': 3, 'min_child_weight': 0.04223350432073095, 'min_split_gain': 0.02848577504214242, 'learning_rate': 0.009275600457696374, 'reg_alpha': 0.0002891790569924796, 'reg_lambda': 0.002416741467994024, 'subsample': 0.7540790375924444, 'colsample_bytree': 0.7132816515204703, 'subsample_freq': 4, 'n_estimators': 940, 'num_leaves': 147, 'max_bin': 301, 'feature_fraction': 0.8323688422687402, 'drop_rate': 0.3886137334721955, 'skip_drop': 0.6967285097125954}. Best is trial 38 with value: 2.263787859316132.


Best trial: 38. Best value: 2.26379: 100%|██████████| 50/50 [18:56<00:00, 22.73s/it]

[I 2025-10-26 14:11:17,473] Trial 49 finished with value: 2.5202830785918096 and parameters: {'boosting_type': 'gbdt', 'max_depth': 3, 'min_child_weight': 0.11038833009158293, 'min_split_gain': 0.061782727686838956, 'learning_rate': 0.0880661702048726, 'reg_alpha': 0.0007680226413252077, 'reg_lambda': 0.17537139329102544, 'subsample': 0.6551447434450544, 'colsample_bytree': 0.9409237687843819, 'subsample_freq': 4, 'n_estimators': 1197, 'num_leaves': 137, 'max_bin': 331, 'feature_fraction': 0.866608472807292}. Best is trial 38 with value: 2.263787859316132.
Best parameters: {'boosting_type': 'gbdt', 'max_depth': 5, 'min_child_weight': 0.5799864138381812, 'min_split_gain': 0.10002020226987698, 'learning_rate': 0.00633897126337854, 'reg_alpha': 1.1721362181810486e-05, 'reg_lambda': 0.0028486856885010254, 'subsample': 0.6639756444298484, 'colsample_bytree': 0.720617007970321, 'subsample_freq': 6, 'n_estimators': 539, 'num_leaves': 133, 'max_bin': 413, 'feature_fraction': 0.925618002124027}

In [36]:
# === 4 Log kết quả tổng hợp ===
task_grad.upload_artifact("Best Parameters", study_grad.best_trial.params)

# Fit lại mô hình với best params
best_params = study_grad.best_trial.params
best_base_model = LGBMRegressor(**best_params)
best_model_grad = MultiOutputRegressor(estimator=best_base_model, n_jobs=-1)
best_model_grad.fit(X_train_sel, y_train)

# Predict
y_pred_val = best_model_grad.predict(X_val_sel)
y_pred_test = best_model_grad.predict(X_test_sel)
y_pred_train = best_model_grad.predict(X_train_sel)

# Evaluate on validation
bestGrad_val_metrics = evaluate_multi_output(y_val, y_pred_val)
print("Best Model - Validation metrics (average):", bestGrad_val_metrics["average"])

# Evaluate on test
bestGrad_test_metrics = evaluate_multi_output(y_test, y_pred_test)
print("Best Model - Test metrics (average):", bestGrad_test_metrics["average"])

# Evaluate on train
bestGrad_train_metrics = evaluate_multi_output(y_train, y_pred_train)
print("Best Model - Train metrics (average):", bestGrad_train_metrics["average"])


Best Model - Validation metrics (average): {'RMSE': 2.3933413529972865, 'R2': 0.7745905147433437, 'MAPE': 8.831995756348775}
Best Model - Test metrics (average): {'RMSE': 1.8683812745306416, 'R2': 0.8345628254448123, 'MAPE': 6.142298263287459}
Best Model - Train metrics (average): {'RMSE': 1.6211098814902147, 'R2': 0.8981529097080241, 'MAPE': 5.7187985480385155}


In [37]:


# 6 Log lên ClearML
val_avg_metrics = bestGrad_val_metrics["average"]
test_avg_metrics = bestGrad_test_metrics["average"]
logger_grad.report_scalar("Best Metrics", "RMSE_val_avg", val_avg_metrics["RMSE"], iteration=0)
logger_grad.report_scalar("Best Metrics", "R2_val_avg", val_avg_metrics["R2"], iteration=0)
logger_grad.report_scalar("Best Metrics", "MAPE_val_avg", val_avg_metrics["MAPE"], iteration=0)
logger_grad.report_scalar("Best Metrics", "RMSE_test_avg", test_avg_metrics["RMSE"], iteration=0)
logger_grad.report_scalar("Best Metrics", "R2_test_avg", test_avg_metrics["R2"], iteration=0)
logger_grad.report_scalar("Best Metrics", "MAPE_test_avg", test_avg_metrics["MAPE"], iteration=0)
logger_grad.report_scalar("Best Metrics", "RMSE_train_avg", train_avg_metrics["RMSE"], iteration=0)
logger_grad.report_scalar("Best Metrics", "R2_train_avg", train_avg_metrics["R2"], iteration=0)
logger_grad.report_scalar("Best Metrics", "MAPE_train_avg", train_avg_metrics["MAPE"], iteration=0)


task_grad.upload_artifact("Best Metrics Detailed", {
    "Validation": bestGrad_val_metrics,
    "Test": bestGrad_test_metrics,
    "Train": bestGrad_train_metrics
})

# Save & upload model
model_path = r'models/Tuning_lgb_model.pkl' 
joblib.dump(best_model_grad, model_path)
print(f"lgb model saved to {model_path}")
task_grad.upload_artifact("Best Model", model_path)

lgb model saved to models/Tuning_lgb_model.pkl


True

In [ ]:
# === Vẽ biểu đồ RMSE theo trial  ===
fig3 = plt.figure(figsize=(7, 4))
plt.plot([t.value for t in study_grad.trials])
plt.xlabel("Trial")
plt.ylabel("RMSE")
plt.title("Optuna RMSE per Trial")

logger_grad.report_matplotlib_figure(
    title="Optuna Performance", 
    series="RMSE Curve", 
    figure=fig3,
    iteration=len(study_grad.trials)
)
plt.close(fig3)


In [ ]:
vis.plot_parallel_coordinate(study_grad)

In [ ]:
vis.plot_slice(study=study_grad)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
                                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': True},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [0.8712339110678275, 0.9014234438413611, 0.899517691011127,
                    0.9421874235023117, 0.9486886936600517, 0.8563558350286024,
                    0.8927541018358549, 0.8661221287254004, 0.8927107788626256,
                    0.947178208272096, 0.8528478291139971, 0.8513553148025303,
                    0.8715112163763844, 0.8755407504889483, 0.8740863874583386,
                    0.9198507707597202, 0.8670559496747424, 0.8846091158324922,
                    0.9163161405482261, 0.8636088734425605, 0.8829910613622213,
                    0.8785098183876235, 0.8627958523743791, 0.8811442422940439,
                    0.8612224802214589, 0.8775530303842521, 0.8865013098893159,
                    0.8685304178685387, 0.9063346246431395, 0.8715684159158977,
                    0.8584211008409289, 0.8753548373649981, 0.8697078520524402,
                    0.8897280760582977, 0.8927707709424861, 0.905747012166398,
                    0.9016933689976901, 0.9130996700033748, 0.9300449091393835,
                    0.901485644292268, 0.9080261965222004, 0.8951874981855131,
                    0.9001956436460746, 0.8979061404922908, 0.8980128982499493,
                    0.9252256398333366, 0.932804862427679, 0.9244336910182035,
                    0.9320163322379106, 0.9407093101733085],
              'xaxis': 'x',
              'y': [2.3608831792247367, 2.2820522525287616, 2.318979126304997,
                    2.2388340980678647, 2.2569534097225974, 2.229492403550998,
                    2.3261397934918158, 2.2148135864928276, 2.318103891151641,
                    2.2547791298803928, 2.252198774143338, 2.237786023098335,
                    2.222565367628073, 2.5307493070512326, 2.2240652745890386,
                    2.29553608521051, 2.2389529783134523, 2.2581816980912883,
                    2.2395378422107144, 2.234900660096892, 2.518379967672704,
                    2.22208513802615, 2.367841016161326, 2.2548665709096385,
                    2.234500408344479, 2.2410339814064644, 2.283454395613842,
                    2.270646837716169, 2.294690368584063, 2.296999378092976,
                    2.42180070090883, 2.222058446044877, 2.267337575540688,
                    2.221377558730074, 2.2312876860289883, 2.217123698579245,
                    2.215990140537614, 2.2257298756246833, 2.225093158793382,
                    2.2352416380844304, 2.2530225886515285, 2.2131069675604977,
                    2.2168445589309247, 2.2251290920895492, 2.222747079077472,
                    2.215792968484861, 2.265995215493276, 2.2340069541869476,
                    2.214915918398305, 2.2250385872599927],
              'yaxis': 'y'},
             {'marker': {'color': [0, 1, 2, 3, 4,

In [ ]:
vis.plot_param_importances(study=study_grad)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [min_impurity_decrease (FloatDistribution):
                                0.0019895667269517617<extra></extra>,
                                min_samples_leaf (IntDistribution):
                                0.001990150838463588<extra></extra>, loss
                                (CategoricalDistribution):
                                0.006198301422825004<extra></extra>, subsample
                                (FloatDistribution):
                                0.007875805982251481<extra></extra>, max_features
                                (CategoricalDistribution):
                                0.009707534715602695<extra></extra>,
                                min_samples_split (IntDistribution):
                                0.009918224176536853<extra></extra>, n_estimators
                                (IntDistribution):
                                0.012472575371803579<extra></extra>, max_depth
                                (IntDistribution):
                                0.013882644785282039<extra></extra>, alpha
                                (FloatDistribution):
                                0.014865501170405214<extra></extra>, max_leaf_nodes
                                (IntDistribution):
                                0.017696264856904596<extra></extra>, learning_rate
                                (FloatDistribution):
                                0.9034034299529733<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [<0.01, <0.01, <0.01, <0.01, <0.01, <0.01, 0.01, 0.01, 0.01,
                       0.02, 0.90],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.0019895667269517617, 0.001990150838463588,
                    0.006198301422825004, 0.007875805982251481,
                    0.009707534715602695, 0.009918224176536853,
                    0.012472575371803579, 0.013882644785282039,
                    0.014865501170405214, 0.017696264856904596, 0.9034034299529733],
              'y': [min_impurity_decrease, min_samples_leaf, loss, subsample,
                    max_features, min_samples_split, n_estimators, max_depth,
                    alpha, max_leaf_nodes, learning_rate]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [ ]:
task_grad.close()